# 1. Configura Kubernetes Authentication

In [ ]:
%env WORKDIR=/tmp/vault
%env VAULT_K8S_NAMESPACE=vault
%env VAULT_HELM_RELEASE_NAME=vault
%env VAULT_SERVICE_NAME=vault-internal 
%env K8S_CLUSTER_NAME=cluster.local 

Importamos la dirección, localización del fichero de la CA de Vault así como el Vault token usando [python-dotenv](https://pypi.org/project/python-dotenv/). De esta forma evitamos filtrar el root token de Vault

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv("/tmp/vault/config.env")

VAULT_TOKEN = os.getenv('VAULT_TOKEN')
VAULT_ADDR = os.getenv('VAULT_ADDR')
VAULT_CACERT = os.getenv('VAULT_CACERT')


Habilitamos Kubernetes Auth Method usando el path por defecto (```kubernetes```)

In [ ]:
! vault auth enable kubernetes

Configuramos Vault para que hable con la API del clúster local usando su propia CA. La SA que usa Vault tiene permisos para verificar los tokens presnetados contra la API de Vault contra la API de Kubernetes.

In [ ]:
%%bash
HOST=$(kubectl get svc kubernetes -o json | jq -r .spec.clusterIP)
PORT=$(kubectl get svc kubernetes -o json | jq -r '.spec.ports.[0].port')
vault write auth/kubernetes/config kubernetes_host=https://$HOST:$PORT

Creamos un role que será usado por las aplicaciones vía VSO para acceder a los secretos de Vault

In [ ]:
%%bash

vault write auth/kubernetes/role/role \
    bound_service_account_names=default \
    bound_service_account_namespaces=test \
    policies=devk8s \
    ttl=10m

Y la política asociada al role que hemos creado en el paso previos

In [ ]:
%%bash
vault policy write devk8s - <<EOF
path "kvv2/*" {
  capabilities = ["read"]
}
EOF

### Creamos un secreto estático en Vault

Habilitamos la engine

In [ ]:
! vault secrets enable -path=kvv2 kv-v2

Creamos un secreto

In [ ]:
! vault kv put kvv2/webapp/config username="static-user" password="static-password"

# 2. Instalando el VSO usando helm. [Referencia](https://developer.hashicorp.com/vault/docs/platform/k8s/vso/installation)

In [ ]:
%%bash
helm repo add hashicorp https://helm.releases.hashicorp.com
helm repo update
# Los recursos se instalaran en el namespace vault-secrets-operator
helm install --version 0.9.1 --create-namespace --namespace vault-secrets-operator vault-secrets-operator hashicorp/vault-secrets-operator

In [ ]:
! kubectl get events -n vault-secrets-operator

In [ ]:
! kubectl get pods -n vault-secrets-operator

### Instalamos los CRDs

[VaultConnection](https://developer.hashicorp.com/vault/docs/platform/k8s/vso/api-reference#vaultconnectionspec)

In [ ]:
%%bash

cat > ${WORKDIR}/vso_crd.yaml <<EOF
---
apiVersion: secrets.hashicorp.com/v1beta1
kind: VaultConnection
metadata:
  namespace: vault-secrets-operator
  name: example
spec:
  address: https://vault.vault.svc.cluster.local:8200
  skipTLSVerify: true

EOF

kubectl apply -f ${WORKDIR}/vso_crd.yaml

Verify VaultConnection is being deployed

In [ ]:
! kubectl describe VaultConnection example -n vault-secrets-operator

[VaultAuth](https://developer.hashicorp.com/vault/docs/platform/k8s/vso/api-reference#vaultauthspec)

In [ ]:
%%bash
cat > ${WORKDIR}/vaultauth_crd.yaml <<EOF
---
apiVersion: secrets.hashicorp.com/v1beta1
kind: VaultAuth
metadata:
  namespace: vault-secrets-operator
  name: example
spec:
  vaultConnectionRef: example
  allowedNamespaces: ["*"]
  method: kubernetes
  mount: kubernetes

  kubernetes:
    # role to use when authenticating to Vault
    role: role
    serviceAccount: default

EOF
kubectl apply -f ${WORKDIR}/vaultauth_crd.yaml


Verify VaultAuth is being deployed

In [ ]:
! kubectl describe VaultAuth example -n vault-secrets-operator

[VaultStaticSecret](https://developer.hashicorp.com/vault/docs/platform/k8s/vso/api-reference#vaultstaticsecretspec)


In [ ]:
! kubectl create ns test

In [ ]:
%%bash
## Support KVv1 and KVv2
cat > ${WORKDIR}/static_secret.yaml <<EOF
---
apiVersion: secrets.hashicorp.com/v1beta1
kind: VaultStaticSecret
metadata:
  namespace: test
  name: example
spec:
  vaultAuthRef: vault-secrets-operator/example
  mount: kvv2
  type: kv-v2
  path: webapp/config
  refreshAfter: 60s
  destination:
    create: true
    name: static-secret1
EOF

kubectl apply -f ${WORKDIR}/static_secret.yaml

Verificamos que el secreto se sincroniza

In [ ]:
! kubectl describe VaultStaticSecret example -n test

Por último veriquemos que podemos leer el secreto (decodificar el base64)

In [ ]:
%%bash
echo "USERNAME: $(kubectl get secret static-secret1 -n test -o json | jq -r .data.username | base64 -d)"
echo "PASSWORD: $(kubectl get secret static-secret1 -n test -o json | jq -r .data.password | base64 -d)"

In [ ]:
%%bash
cat > ${WORKDIR}/mypod.yaml <<EOF
apiVersion: v1
kind: Pod
metadata:
  name: mypod
  namespace: test
spec:
  containers:
  - name: mypod
    image: nginx
    env:
    - name: USERNAME
      valueFrom:
        secretKeyRef:
          name: static-secret1
          key: username
    - name: PASSWORD
      valueFrom:
        secretKeyRef:
          name: static-secret1
          key: password
EOF

kubectl apply -f ${WORKDIR}/mypod.yaml
sleep 10



In [ ]:
! kubectl exec mypod -n test -- env | grep -E 'USERNAME|PASSWORD'

### Dynamic Secrets

Actualizar la políticas con permisos de lectura al path de la database

In [ ]:
%%bash

vault policy write devk8s - <<EOF
path "kvv2/*" {
  capabilities = ["read"]
}
path "database/creds/readonly" {
  capabilities = [ "read"]
}
EOF

[VaultDynamicSecret](https://developer.hashicorp.com/vault/docs/platform/k8s/vso/api-reference#vaultdynamicsecretspec)

In [ ]:
%%bash

cat > ${WORKDIR}/dynamic_secret.yaml <<EOF
---
apiVersion: secrets.hashicorp.com/v1beta1
kind: VaultDynamicSecret
metadata:
  namespace: test
  name: db-secret
spec:
  vaultAuthRef: vault-secrets-operator/example
  mount: database
  path: creds/readonly
  destination:
    create: true
    name: db-secret
EOF

kubectl apply -f ${WORKDIR}/dynamic_secret.yaml


Verificar que el secreto ha sido sincronizado

In [ ]:
%%bash
sleep 5
kubectl get secret db-secret -n test -o yaml

Ahora decodificando el base64

In [ ]:
%%bash
echo "USERNAME: $(kubectl get secret db-secret -n test -o json | jq -r .data.username | base64 -d)"
echo "PASSWORD: $(kubectl get secret db-secret -n test -o json | jq -r .data.password | base64 -d)"

### Montemos el secreto en un POD

In [ ]:
%%bash
cat > ${WORKDIR}/mypod_db.yaml <<EOF
apiVersion: v1
kind: Pod
metadata:
  name: mypoddb
  namespace: test
spec:
  containers:
  - name: mypod
    image: redis
    volumeMounts:
    - name: foo
      mountPath: "/etc/foo"
      readOnly: true
  volumes:
  - name: foo
    secret:
      secretName: db-secret
      optional: true
EOF


# Despliega el POD
kubectl apply -f ${WORKDIR}/mypod_db.yaml

Verifiquemos que el secreto se monta en el pod

In [ ]:
%%bash
# Espera a que despliegue
sleep 10 
# Chequea secretos
kubectl exec mypoddb -n test  -- ls /etc/foo/
echo ""
echo "------"
kubectl exec mypoddb -n test -- cat /etc/foo/_raw

Si esperamos dos minutos (por la configuración del role) veremos un nuevo secreto en el volumen del POD

In [ ]:
%%bash
sleep 120 
kubectl exec mypoddb -n test -- cat /etc/foo/_raw